In [ ]:
from keras.utils import multi_gpu_model
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from time import time
from keras.layers import Dense
import keras
from helper_functions import get_filenames
from keras.utils import multi_gpu_model

In [ ]:
filenames = get_filenames("D:/dataset3/photos")

In [ ]:
data = []
f = open('dataset3/photo.json', 'r')
for i in tqdm(range(200000)):
    line = json.loads(f.readline())
    data.append((line["photo_id"], line["business_id"], line["label"]))
f.close()

In [ ]:
ResNet_model = ResNet50(include_top=True, weights='imagenet')

In [ ]:
ResNet_model = ResNet50(include_top=True, weights='imagenet')
output = Dense(5, activation="softmax")(ResNet_model.output)
model = Model(inputs = ResNet_model.input, outputs=output)
for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
lbs = {"drink":[1,0,0,0,0], "food":[0,1,0,0,0], "menu":[0,0,1,0,0], "inside":[0,0,0,1,0], "outside":[0,0,0,0,1]}

In [ ]:
data_dict = dict()
for i in data:
    data_dict[i[0]] = (i[1], i[2])

In [ ]:
def batch_generator(filenames):
    while True:
        
        X = []
        Y = []
        
        for f in filenames:
                        
            img_id = f[-f[::-1].index("\\"):-f[::-1].index(".")-1]
            
            Y.append(lbs[data_dict[img_id][1]])
            
            img_path = f
            img = image.img_to_array(image.load_img(img_path, target_size=(224, 224)))
            
            X.append(img)
            
            if len(X)==40:
                
                yield np.array(X), np.array(Y)
                
                X = []
                Y = []
        
        yield np.array(X), np.array(Y)

In [ ]:
train_filenames = filenames[:180000]
test_filenames = filenames[180000:199000]
validation_filenames = filenames[199000:]
train_gen = batch_generator(filenames)
test_gen = batch_generator(test_filenames)
validation_gen = batch_generator(validation_filenames)

In [ ]:
for i in tqdm(range(15000)):
    batch = next(train_gen)
    model.train_on_batch(batch[0], batch[1])
    if (i+1)%1000==0:
        for i in range(25):
            batch = next(validation_gen)
            print(model.evaluate(batch[0], batch[1]))

In [ ]:
model.save_weights("weights5.h5")